<h1>Coursera Data Science Capstone Project Part - 1<h1>
<h4>This notebook will be used to implement the Capstone projects titled <b><i>"The battle of neighbourhoods"</i></b></h4>

<h3>Segmenting and Clustering Neighborhoods in Toronto</h3>

In [3]:
import numpy as np
import pandas as pd
import requests
import lxml.html as lh

print('Libraries imported')

Libraries imported


<h4>Scraping the Toronto Wikipedia Page using lxml parser from Beautiful Soap</h4>

In [0]:
#Toronto Wikipedia link
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#getting content from the html page
page_content = lh.fromstring(requests.get(url).content)
#accessing the table data
table_data = page_content.xpath("//tr")

<h4>Generating Toronto Neighbourhood data from scrapped data as a Python list</h4>

In [5]:
toronto_data=[]
col_ind=0

#Creating header with empty list to add list of values under each column, replace new line character
for th in table_data[0]:
    col_ind+=1
    header=str(th.text_content()).replace('\n','')
    print (col_ind,header)
    toronto_data.append((header,[]))
    #print(toronto_data)

1 Postcode
2 Borough
3 Neighbourhood


In [0]:
# range starts from 1, since header is handled
for td in range(1,len(table_data)):
    row=table_data[td]
    
    #Exclude if the row length is not 3
    if len(row) != 3:
      break
    
    col_ind=0
  
    #Iterating through each element of the row, replace new line character
    for val in row.iterchildren():
        data=str(val.text_content()).replace('\n','')
        #Append the data after header
        toronto_data[col_ind][1].append(data)
        #Increment i for next column
        col_ind+=1

#print(toronto_data)
#print(len(toronto_data))
#print([len(rows) for (i,rows) in toronto_data])

<h4>Creating a Pandas Data frame from the List of headers and rows of Toronto Neighbourhood data</h4>

In [7]:
#Creating a Pandas Data Frame
toronto_struct={header:rows for (header,rows) in toronto_data}
toronto_df=pd.DataFrame(toronto_struct)

#Dropping rows with Borough value of Not assigned and resetting the index
toronto_df = toronto_df.drop(toronto_df.index[toronto_df['Borough'] == 'Not assigned']).reset_index(drop = True)

#print(toronto_df.shape)
toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


<h4>Further Cleaning the Toronto Neighbourhood data</h4>

In [8]:
#Grouping Neighbourhoods by Postal codes
toronto_df = toronto_df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()

#Replacing Neighbourhood with Not Assigned with name of Borough
toronto_df.loc[toronto_df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = toronto_df['Borough']

#print(toronto_df[toronto_df['Borough'] == 'Queen\'s Park'])
toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h2>Final Dimensions of the Data Frame, after Cleaning and Grouping the data</h2>

In [9]:
toronto_df.shape

(103, 3)

In [0]:
#Creating a csv file of the data to use in Part - 2 of the project
toronto_df.to_csv(r'toronto_data.csv', index=False)